## Fast Text 함수

# 환경설정

In [ ]:
# 패키지 영구 저장
import os, sys

# 구글 마운트 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
import re
import pickle
from collections import Counter
from wordcloud import WordCloud
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


plt.rc('font', family='NanumGothic')            # matplotllib 한글 문장
mpl.rcParams['axes.unicode_minus'] = False      # minus 처리

file_path = '/content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/'

# 데이터 불러오기

In [ ]:
# 데이터 열기 
with open(file_path + 'kiwi_dataframe.pkl', 'rb') as f:
    talk_df = pickle.load(f)

talk_df[['문장','형태소','MORPHS']].loc[20:30,:]

,문장,형태소,MORPHS
20,의사들은 좋겠다 돈 많이 벌어서,"[(의사, NNG), (들, XSN), (좋다, VA), (겠, EP), (다, E...","[의사, 들, 좋다, 돈, 많이, 벌다]"
21,근데 병원에 있으면 너무 우울할듯,"[(근데, MAJ), (병원, NNG), (있다, VV), (으면, EC), (너무...","[근데, 병원, 있다, 너무, 우울, 하, 듯]"
22,1년 반 동안 한 거라곤 세금 퍼주기랑 거리두기밖에 없네,"[(1, SN), (년, NNB), (반, NNG), (동안, NNG), (하다, ...","[1, 년, 반, 동안, 하다, 거, 이다, 세금, 퍼주기, 거리두기, 없다]"
23,지금껏 포퓰리즘 정책밖에 없음,"[(지금껏, MAG), (포퓰리즘, NNG), (정책, NNG), (없다, VA),...","[지금껏, 포퓰리즘, 정책, 없다]"
24,난 그래도 좋은 사람이였구나,"[(나, NP), (그렇다, VA), (어도, EC), (좋다, VA), (은, E...","[나, 그렇다, 좋다, 사람, 이다]"
25,비록 여돌가지고 딸치고 놀앗지만 여자에 미쳐서 남자 패지는 않앗으니까 당당하게 살겠...,"[(비록, MAG), (여돌, NNP), (가지다, VV), (고, EC), (딸,...","[비록, 여돌, 가지다, 딸, 놀다, 여자, 미치다, 남자, 패다, 않다, 당당, ..."
26,쟤 이번에 트레이너 회원수 랭킹 1위 찍었다며?,"[(쟤, NP), (이번, NNG), (트레이너, NNP), (회원, NNG), (...","[쟤, 이번, 트레이너, 회원, 수, 랭킹, 1, 위, 찍다]"
27,기생오래비같이 뺀질반반해서 기지배들 취향엔 제격일 뿐 ㅉㅉ,"[(기생오래비, NNP), (같이, MAG), (빼다, VV), (ᆫ, ETM), ...","[기생오래비, 같이, 빼다, 질, 반반, 하, 기지배, 들, 취향, 제격, 이다, ..."
28,쟤는 음악으로 성공하기가 쉬운줄 아나?,"[(쟤, NP), (음악, NNG), (성공, NNG), (하, XSV), (기, ...","[쟤, 음악, 성공, 하, 쉽다, 줄, 알다]"
29,그니까 음악한다고 집에만 박혀서 부모님 등골빨아먹는게 참 재수없다,"[(그니까, MAJ), (음악, NNG), (하, XSV), (ᆫ다고, EC), (...","[그니까, 음악, 하, 집, 박히다, 부모님, 등골, 빨아먹다, 것, 참, 재수, 없다]"


# 집합으로 표현하기

In [ ]:
def ExtractText(series):
    text_lst = []
    for sent in series:
        text_lst.extend(sent)
    return text_lst 

text_none = ExtractText(talk_df.loc[talk_df['분류']=='비악플',:]['형태소'])
text_advisory = ExtractText(talk_df.loc[talk_df['분류']=='권고',:]['형태소'])
text_caution = ExtractText(talk_df.loc[talk_df['분류']=='주의',:]['형태소'])
text_warning = ExtractText(talk_df.loc[talk_df['분류']=='경고',:]['형태소'])

In [ ]:
set_none = set(text_none)
set_advisory = set(text_advisory)
set_caution = set(text_caution)
set_warning = set(text_warning)

In [ ]:
print(f'"비악플"의 형태소는 {len(set_none)} 개 입니다.')
print(f'"권고(비난/차별/혐오)"의 형태소는 {len(set_advisory)} 개 입니다.')
print(f'"주의(욕설/범죄)"의 형태소는 {len(set_caution)} 개 입니다.')
print(f'"경고(선정/폭력)"의 형태소는 {len(set_warning)} 개 입니다.')

"비악플"의 형태소는 25094 개 입니다.
"권고(비난/차별/혐오)"의 형태소는 27310 개 입니다.
"주의(욕설/범죄)"의 형태소는 10842 개 입니다.
"경고(선정/폭력)"의 형태소는 12985 개 입니다.


In [ ]:
set1 = set_none & set_advisory & set_caution & set_warning
set12 = set_advisory & set_caution & set_warning - set1
set2 = set_none - set_advisory - set_caution - set_warning
set3 = (set_none & set_caution) - set_advisory - set_warning
set4 = (set_none & set_advisory) - set_caution - set_warning 
set5 = (set_none & set_warning) - set_advisory - set_caution 
set6 = (set_none & set_caution) - set_warning - set3 
set7 = (set_none & set_advisory) - set1 - set4 - set6
set8 = (set_none & set_warning) - set1 - set5 - set7 
set9 = (set_advisory & set_caution) - set1 - set6 - set12
set10 = (set_advisory & set_warning) - set1 - set7 - set12 
set11 = (set_caution & set_warning) - set1 - set8 - set12
set13 = set_advisory - set_caution - set_warning - set_none
set14 = set_caution - set_warning - set_advisory - set_none
set15 = set_warning - set_advisory - set_caution - set_none

In [ ]:
print(f'비악플, 권고, 주의, 경고에서 모두 쓰인 형태소는 {len(set1)} 개 입니다.')
print(f'비악플에서만 쓰인 형태소는 {len(set2)} 개 입니다.')
print(f'권고에서만 쓰인 형태소는 {len(set13)} 개 입니다.')
print(f'주의에서만 쓰인 형태소는 {len(set14)} 개 입니다.')
print(f'경고에서만 쓰인 형태소는 {len(set15)} 개 입니다.')
print(f'비악플과 주의에서만 쓰인 형태소는 {len(set3)} 개 입니다.')
print(f'비악플과 권고에서만 쓰인 형태소는 {len(set4)} 개 입니다.')
print(f'비악플과 경고에서만 쓰인 형태소는 {len(set5)} 개 입니다.')
print(f'비악플, 권고, 주의에서만 쓰인 형태소는 {len(set6)} 개 입니다.')
print(f'비악플, 권고, 경고에서만 쓰인 형태소는 {len(set7)} 개 입니다.')
print(f'비악플, 주의, 경고에서만 쓰인 형태소는 {len(set8)} 개 입니다.')
print(f'권고, 주의에서만 쓰인 형태소는 {len(set9)} 개 입니다.')
print(f'권고, 경고에서만 쓰인 형태소는 {len(set10)} 개 입니다.')
print(f'주의, 경고에서만 쓰인 형태소는 {len(set11)} 개 입니다.')
print(f'권고, 주의, 경고(악플 모두)에서 쓰인 형태소는 {len(set12)} 개 입니다.')

비악플, 권고, 주의, 경고에서 모두 쓰인 형태소는 5638 개 입니다.
비악플에서만 쓰인 형태소는 7106 개 입니다.
권고에서만 쓰인 형태소는 8387 개 입니다.
주의에서만 쓰인 형태소는 1584 개 입니다.
경고에서만 쓰인 형태소는 2037 개 입니다.
비악플과 주의에서만 쓰인 형태소는 345 개 입니다.
비악플과 권고에서만 쓰인 형태소는 6319 개 입니다.
비악플과 경고에서만 쓰인 형태소는 700 개 입니다.
비악플, 권고, 주의에서만 쓰인 형태소는 1893 개 입니다.
비악플, 권고, 경고에서만 쓰인 형태소는 2988 개 입니다.
비악플, 주의, 경고에서만 쓰인 형태소는 105 개 입니다.
권고, 주의에서만 쓰인 형태소는 724 개 입니다.
권고, 경고에서만 쓰인 형태소는 964 개 입니다.
주의, 경고에서만 쓰인 형태소는 156 개 입니다.
권고, 주의, 경고(악플 모두)에서 쓰인 형태소는 397 개 입니다.


# 단독을 키워드로 유사도 분석하기(set13, set14, set15)

## Fast Text 함수

In [ ]:
#from gensim.models import FastText
#from gensim.models import Word2Vec

#model = FastText(sentences=talk_df['MORPHS'], window=5, min_count=5, workers=4, sg=1)

In [ ]:
#print(len(list(model.wv.vocab)))

In [ ]:
#set13_nouns = [pos[0] for pos in set13 if pos[1][0]=='N']
#set14_nouns = [pos[0] for pos in set14 if pos[1][0]=='N']
#set15_nouns = [pos[0] for pos in set15 if pos[1][0]=='N']
#print(set13_nouns)
#print(set14_nouns)
#print(set15_nouns)

## GloVe 함수

In [ ]:
pip install glove_python_binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.9/974.9 KB 36.5 MB/s eta 0:00:00


In [ ]:
# import 해서 불러오기
from glove import Corpus, Glove
import numpy as np
import csv

# corpus 생성
corpus = Corpus()
corpus.fit(talk_df['MORPHS'].tolist(), window=20)

# model
glove = Glove(no_components=128, learning_rate=0.01)     # 128차원으로 임베딩된 벡터로 나온도록 설정
%time glove.fit(corpus.matrix, epochs=50, no_threads=4, verbose=False)   # 위에서 만들어준 corpus를 matrix 형태로 넣어줌  # Wall time: 2min 36s
glove.add_dictionary(corpus.dictionary)

# save
glove.save(file_path +'/kiwi_glove_w20_epoch50.model_2')

# load glove
glove_model = Glove.load(file_path + '/kiwi_glove_w20_epoch50.model_2')  #glvoe 모델이 단어들에 대해서 어떤 임베딩 벡터를 내보내는지 확인

# word dict 생성
word_dict = {}
for word in  glove_model.dictionary.keys():
    word_dict[word] = glove_model.word_vectors[glove_model.dictionary[word]]
print('[Success !] Lengh of word dict... : ', len(word_dict))

# save word_dict
with open(file_path + '/glove_word_dict.pickle', 'wb') as f:
    pickle.dump(word_dict, f)
print('[Success !] Save word dict!...')

def sent2vec_glove(tokens, embedding_dim=128):
    '''문장 token 리스트를 받아서 임베딩 시킨다.'''
  
    size = len(tokens)
    matrix = np.zeros((size, embedding_dim))
    word_table = word_dict     # glove word_dict

    for i, token in enumerate(tokens):
        vector = np.array([
            word_table[t] for t in token
            if t in word_table
        ])

        if vector.size != 0:
            final_vector = np.mean(vector, axis=0)
            matrix[i] = final_vector

    return matrix

CPU times: user 5min 20s, sys: 451 ms, total: 5min 21s
Wall time: 3min 22s
[Success !] Lengh of word dict... :  35337
[Success !] Save word dict!...


In [ ]:
# print(len(list(model.wv.vocab)))

In [ ]:
set13_nouns = [pos[0] for pos in set13 if pos[1][0]=='N']
set14_nouns = [pos[0] for pos in set14 if pos[1][0]=='N']
set15_nouns = [pos[0] for pos in set15 if pos[1][0]=='N']
print(set13_nouns)
print(set14_nouns)
print(set15_nouns)

['초급', '짓이지', '말뽄새', '몰랑', '바라기', '전지훈련', '호뽑뽑요', '상응', '콘', '착짱죽짱', '디지시길', '듸지지', '렉카충', '부엌일', '어디가냐', '할아비', '돼냐', '연속극', '해충구제', '이민자', '배끼', '차후', '샌들', '심혈관질환', '삼촌팬', '개빻았누', '하쉴', '뚱돼지게', '골짜기', '외노', '찐따', '풍뎅이', '생물체', '백따리', '쩜', '흑인놈', '이희진', '슬하', '대의명분', '시나몬', '안건', '소나타', '오지구여', '빚보증', '토착민', '아웃팅', '쩌어억', '앰탈이네', '개역겹네', '관심팔이', '파냐', '페스카마호', '머하냐', '둔재', '인간애', '빠따충', '효율성', '특공', '결집', '가기', '피싸듯', '연구소', '부설', '걸요', '세이', '치셈', '플레이팅', '빠순', '놔둬라', '아무개', '쌍하타치됨', '지끄리', '버선발', '소시오패스야', '흥선대원군', '플라톤', '박인비', '가타부타', '만덕산', '한가위', '호록록', '제', '누렁이', '필살기', '브녀충', '도덕심', '폐품', '프린트', '선도', '껌댕이', '나오노', '플레인', '임팩트', '났누', '쥰내', '뺀질', '창조물', '머하', '주번', '활동비', '할배할녀', '반일', '땅부자', '코호트', '대행자', '조국', '추수', '띵가띵가', '따흐', '전담충', '인정사정', '동명', '동남아시아인', '좌파때문', '의약', '밭일', '의절', '원숭아', '됐쥬', '점유', '해결안', '덫', '티내죠', '쫑알쫑알', '인싱', '견본', '깝침', '아스날', '주워다', '꾸졌지', '짱나고', '구매관재', '술고래', '호구', '필로폰', '블랙컨슈머', '조총련', '휴무일', '윗입술', '철갑', '탱커', '간뎅이', '주당', '코스메', '부쒓네', '

## 키워드 추출하기 

In [ ]:
# 방법 : 키워드 리스트를 넣었을 때 상위 n개 가져오기
# input: keywordlist=[(keyword1, tag1), (keyword2, tag2), ...]
def ExtractSimilar(word_lst, n, model_name):
    stopwords = ['다']
    sim_lst = []
    sim_set = set()
    for key, tag in word_lst:
        sim = model_name.most_similar(key, number=n)
        for word, p in sim:
            if word[0].isdigit()==False and len(word) > 1 and word not in sim_set:
              if p > 0.8 and word[-1] not in stopwords:
                sim_lst.append([word, key, round(p,4), tag, len(word)])
                sim_set.add(word)

    sim_df = pd.DataFrame(sim_lst, columns=['keyword', 'search', 'similarity', 'tag', 'len'])
    sim_df.sort_values(by=['search','keyword','len','similarity'], inplace=True)
    sim_df.reset_index(drop=True, inplace=True)
    
    return sim_df


## 전에 했던거 편집

In [ ]:
dic_df = pd.read_csv(file_path + '[은지,편집] GloVe 혐오단어 분류.csv', encoding='CP949')

dic_df

,keyword,빈도기반 tag,비악플,선정,폭력,혐오
0,AV,선정,14,17,0,6
1,ㄱㅊ,선정,8,14,8,7
2,가난뱅이,혐오,2,0,0,12
3,가랭이,선정,1,15,0,1
4,가슴,선정,211,496,14,146
...,...,...,...,...,...,...
1254,방구석,혐오,27,4,25,205
1255,꼴깝,혐오,2,1,6,19
1256,엄살,혐오,9,1,2,28
1257,열라,혐오,3,3,4,18


In [ ]:
cat_lst = ['비악플','권고','주의','경고']
    
cnt_lst = []
print('Start!', end=' ')
i = 0
for key in dic_df['keyword'].tolist():
    counts = talk_df.loc[talk_df['MORPHS'].apply(lambda x: key in x),['분류']].value_counts()
    counts = dict([(cat[0], cnt) for cat, cnt in counts.items()])

    if counts == {}:
      max_cat = 0
    else:
      max_cat = list(counts.keys())[0]
    # print(counts)
    if len(counts) != len(cat_lst):
        no_cat = [cat for cat in cat_lst if cat not in counts.keys()]
        for cat in no_cat:
            counts[cat] = 0
    
    cnt_lst.append([max_cat] + [cnt for cat, cnt in sorted(counts.items())])

    # 메세지 출력 
    i += 1
    if i % 100 == 0:
        print(f'> {i}', end=' ')
    if i % 1000 == 0:
        print()
print('End!')

cnt_df = pd.DataFrame(cnt_lst, columns=['빈도기반 tag'] + cat_lst)
dic_df = pd.concat([dic_df, cnt_df], axis=1)
dic_df

Start! > 100 > 200 > 300 > 400 > 500 > 600 > 700 > 800 > 900 > 1000 
> 1100 > 1200 End!


,keyword,빈도기반 tag,비악플,선정,폭력,혐오,빈도기반 tag,비악플,권고,주의,경고,빈도기반 tag,비악플,권고,주의,경고
0,AV,선정,14,17,0,6,경고,17,6,14,0,경고,17,6,14,0
1,ㄱㅊ,선정,8,14,8,7,경고,20,7,8,2,경고,20,7,8,2
2,가난뱅이,혐오,2,0,0,12,권고,0,12,2,0,권고,0,12,2,0
3,가랭이,선정,1,15,0,1,경고,15,1,1,0,경고,15,1,1,0
4,가슴,선정,211,496,14,146,경고,499,144,209,10,경고,499,144,209,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,방구석,혐오,27,4,25,205,권고,11,260,36,28,권고,11,260,36,28
1255,꼴깝,혐오,2,1,6,19,권고,1,21,2,8,권고,1,21,2,8
1256,엄살,혐오,9,1,2,28,권고,2,28,9,1,권고,2,28,9,1
1257,열라,혐오,3,3,4,18,권고,4,17,3,3,권고,4,17,3,3


In [ ]:
dic_df.to_csv(file_path + 'eunji_dic.csv', index=False)

### 1) 권고(Advisory, 비난/혐오/차별)

In [ ]:
# 반복 함수
advisory_lst=[(noun, '권고') for noun in set13_nouns if len(noun) > 1 and noun in model.wv.vocab.keys()]
rep = 5
print('Start!', end=' ')
for i in range(rep):
    dic_df = ExtractSimilar(advisory_lst, n=10, model_name=model)
    print(f'> {dic_df.shape[0]}', end=' ')

    advisory_lst=[(noun, '권고') for noun in dic_df['keyword'] if len(noun) > 1 and noun in model.wv.vocab.keys()]
print('End!')
dic_df

In [ ]:
dic_df = pd.read_csv(file_path + 'eunji_dic.csv')

In [ ]:
# 새로운 태그 만들기
cat_lst = ['비악플','권고','주의','경고']
    
cnt_lst = []
print('Start!', end=' ')
i = 0
for key in dic_df['keyword'].tolist():
    counts = talk_df.loc[talk_df['MORPHS'].apply(lambda x: key in x),['분류']].value_counts()
    counts = dict([(cat[0], cnt) for cat, cnt in counts.items()])
    max_cat = list(counts.keys())[0]
    # print(counts)
    if len(counts) != len(cat_lst):
        no_cat = [cat for cat in cat_lst if cat not in counts.keys()]
        for cat in no_cat:
            counts[cat] = 0
    
    cnt_lst.append([max_cat] + [cnt for cat, cnt in sorted(counts.items())])

    # 메세지 출력 
    i += 1
    if i % 200 == 0:
        print(f'> {i}', end=' ')
    if i % 2000 == 0:
        print()
print('End!')

cnt_df = pd.DataFrame(cnt_lst, columns=['빈도기반 tag'] + cat_lst)
dic_df = pd.concat([dic_df, cnt_df], axis=1)
dic_df

### 2) 주의(Caution, 욕설/범죄)

In [ ]:
# 반복 함수
caution_lst=[(noun, '주의') for noun in set14_nouns if len(noun) > 1 and noun in model.wv.vocab.keys()]
rep = 5
print('Start!', end=' ')
for i in range(rep):
    dic_df = ExtractSimilar(caution_lst, n=10, model_name=model)
    print(f'> {dic_df.shape[0]}', end=' ')

    caution_lst=[(noun, '주의') for noun in dic_df['keyword'] if len(noun) > 1 and noun in model.wv.vocab.keys()]
print('End!')
dic_df

In [ ]:
# 새로운 태그 만들기
cat_lst = ['비악플','권고','주의','경고']
    
cnt_lst = []
print('Start!', end=' ')
i = 0
for key in dic_df['keyword'].tolist():
    counts = talk_df.loc[talk_df['MORPHS'].apply(lambda x: key in x),['분류']].value_counts()
    counts = dict([(cat[0], cnt) for cat, cnt in counts.items()])
    max_cat = list(counts.keys())[0]
    # print(counts)
    if len(counts) != len(cat_lst):
        no_cat = [cat for cat in cat_lst if cat not in counts.keys()]
        for cat in no_cat:
            counts[cat] = 0
    
    cnt_lst.append([max_cat] + [cnt for cat, cnt in sorted(counts.items())])

    # 메세지 출력 
    i += 1
    if i % 200 == 0:
        print(f'> {i}', end=' ')
    if i % 2000 == 0:
        print()
print('End!')

cnt_df = pd.DataFrame(cnt_lst, columns=['빈도기반 tag'] + cat_lst)
dic_df = pd.concat([dic_df, cnt_df], axis=1)
dic_df

### 3) 경고(Warning, 선정/폭력)

In [ ]:
# 반복 함수
warning_lst=[(noun, '경고') for noun in set15_nouns if len(noun) > 1 and noun in model.wv.vocab.keys()]
rep = 5
print('Start!', end=' ')
for i in range(rep):
    dic_df = ExtractSimilar(warning_lst, n=10, model_name=model)
    print(f'> {dic_df.shape[0]}', end=' ')

    warning_lst=[(noun, 경고') for noun in dic_df['keyword'] if len(noun) > 1 and noun in model.wv.vocab.keys()]
print('End!')
dic_df

In [ ]:
# 새로운 태그 만들기
cat_lst = ['비악플','권고','주의','경고']
    
cnt_lst = []
print('Start!', end=' ')
i = 0
for key in dic_df['keyword'].tolist():
    counts = talk_df.loc[talk_df['MORPHS'].apply(lambda x: key in x),['분류']].value_counts()
    counts = dict([(cat[0], cnt) for cat, cnt in counts.items()])
    max_cat = list(counts.keys())[0]
    # print(counts)
    if len(counts) != len(cat_lst):
        no_cat = [cat for cat in cat_lst if cat not in counts.keys()]
        for cat in no_cat:
            counts[cat] = 0
    
    cnt_lst.append([max_cat] + [cnt for cat, cnt in sorted(counts.items())])

    # 메세지 출력 
    i += 1
    if i % 200 == 0:
        print(f'> {i}', end=' ')
    if i % 2000 == 0:
        print()
print('End!')

cnt_df = pd.DataFrame(cnt_lst, columns=['빈도기반 tag'] + cat_lst)
dic_df = pd.concat([dic_df, cnt_df], axis=1)
dic_df